<a href="https://colab.research.google.com/github/singwang-cn/The_first_pattern_recognition/blob/master/ex8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn import svm

In [176]:
class support_vector_machine():
  def __init__(self, kernel='rbf', sigma=4, c=10):
    self.kernel = kernel
    self.sigma = sigma
    self.c = c
    self.w = None
    self.b = None
    self.l = None
    self.t = None
    self.H = None
    self.alpha = None
    self.cons = (
      {'type': 'eq', 'fun': lambda x: np.dot(x, self.t)}
    )
  def obj_func(self, alpha):
    return 1/2*np.dot(np.dot(alpha, self.H), alpha)-np.sum(alpha)
  def fit(self, X, Y):
    n, d = X.shape
    self.l = X
    self.t = np.where(Y==0, -1, 1)
    self.H = np.zeros((n, n))
    for i in range(n):
      for j in range(n):
        self.H[i][j] = self.t[i]*self.t[j]*self.rbf_kernel(X[i], X[j])
    self.alpha = minimize(self.obj_func, 
                        x0=np.random.rand(n), bounds=((0, self.c),)*n,
                        constraints=self.cons, method="SLSQP").x
  def predict(self, x):
    n = self.alpha.shape[0]
    result = np.zeros(n)
    for i in range(n):
      result[i] = self.alpha[i]*self.t[i]*self.rbf_kernel(self.l[i], x)
    if np.sum(result) > 0:
      return 1
    else:
      return 0
  def rbf_kernel(self, u, v):
    norm = np.linalg.norm(u-v, ord=2)
    return np.exp(-1/(2*self.sigma)*(norm**2))
    

In [188]:
data_path = '/content/drive/My Drive/Dataset/diabetes.csv'
pima = pd.read_csv(data_path)
pima_label = np.array(pima['Outcome'].tolist())
pima_nolabel = pima[['Glucose', 'BMI']]
pima_data = ((pima_nolabel-pima_nolabel.mean())/pima_nolabel.std()).values

In [190]:
svm = support_vector_machine()
svm.fit(pima_data, pima_label)

In [205]:
correct = 0
result = np.zeros(pima_label.shape[0])
for i in range(pima_label.shape[0]):
  result[i] = svm.predict(pima_data[i])
  if result[i] == pima_label[i]:
    #print(svm.predict(pima_data[i]), pima_label[i])
    correct += 1
  #else:
    #print(i)
print(correct/pima_label.shape[0])

0.765625


In [202]:
pima_label[642]

1

In [203]:
svm.predict(pima_data[642])

0